# Notebook - Electron Analyzer

This notebook takes CMS OpenData nanoAOD files, applies some selection and make few simple plots. 

Let's first load the libraries:

In [1]:
import asyncio
import logging
import os
import time

import vector; vector.register_awkward()
import awkward as ak
from coffea import processor
from coffea.nanoevents import transforms
from coffea.nanoevents.methods import base, vector
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import hist
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import uproot

import pickle

In [2]:
#ifile = uproot.open("root://eosuser.cern.ch//eos/user/a/algomez/tmpFiles/opendata_files/SingleElectron/cmsopendata2015_Run2015D_SingleElectron_MINIAOD_08Jun2016-v1_21.root")
#ifile["Events"].keys()

events = NanoEventsFactory.from_root(
    #"https://xrootd-local.unl.edu:1094//store/user/AGC/nanoAOD/TT_TuneCUETP8M1_13TeV-powheg-pythia8/cmsopendata2015_ttbar_19980_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext3-v1_00000_0000.root",
    uproot.open("root://eosuser.cern.ch//eos/user/a/algomez/tmpFiles/opendata_files/SingleElectron/cmsopendata2015_Run2015D_SingleElectron_MINIAOD_08Jun2016-v1_21.root"),
    schemaclass=NanoAODSchema.v6,
    metadata={"dataset": "TT"},
).events()
#print(ak.count(events.Jet.pt,axis=1)>=2)

/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:193: RuntimeWarning: Missing cross-reference index for FatJet_subJetIdx1 => SubJet
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:193: RuntimeWarning: Missing cross-reference index for FatJet_subJetIdx2 => SubJet
  warnings.warn(


For future use, let's define some global configuration

In [3]:
### GLOBAL CONFIGURATION

DATA = "SingleElectron"  ### or SingleElectron

# input files per process, set to e.g. 10 (smaller number = faster)
N_FILES_MAX_PER_SAMPLE = 10

### BENCHMARKING-SPECIFIC SETTINGS

# chunk size to use
CHUNKSIZE = 500_000

# metadata to propagate through to metrics
CORES_PER_WORKER = 2  # does not do anything, only used for metric gathering (set to 2 for distributed coffea-casa)

# scaling for local setups with FuturesExecutor
NUM_CORES = 4

NanoAOD datasets are stored in `data/ntuples_nanoaod.json` folder. This json file contains information about the number of events, process and systematic. The following function reads the json file and returns a dictionary with the process to run.

In [4]:
def construct_fileset(n_files_max_per_sample,
                      dataset="SingleElectron",
                      onlyNominal=False,
                      ntuples_json="ntuples_nanoaod_V2.json"):
    # using https://atlas-groupdata.web.cern.ch/atlas-groupdata/dev/AnalysisTop/TopDataPreparation/XSection-MC15-13TeV.data
    # for reference
    # x-secs are in pb
    xsec_info = {
        "ttbar": 831.76, ###396.87 + 332.97, # nonallhad + allhad, keep same x-sec for all
        #"single_top_s_chan": 2.0268 + 1.2676,
        #"single_top_t_chan": (36.993 + 22.175)/0.252,  # scale from lepton filter to inclusive
        "single_top_tW": 35.6 + 35.6, #37.936 + 37.906,
        "wjets": 61526.7, ##61457 * 0.252,  # e/mu+nu final states
        "tttt" : 0.009, 
        "data": None
    }

    # list of files
    with open(ntuples_json) as f:
        file_info = json.load(f)

    # process into "fileset" summarizing all info
    fileset = {}
    for process in file_info.keys():
        if process == "data":
            file_list = file_info[process][dataset]["files"]
            if n_files_max_per_sample != -1:
                #file_list = file_list[:int(n_files_max_per_sample/10)]  # use partial set of samples
                file_list = file_list[:]  # use partial set of samples

            file_paths = [f["path"] for f in file_list]
            metadata = {"process": "data", "xsec": 1}
            fileset.update({"data": {"files": file_paths, "metadata": metadata}})
            

        for variation in file_info[process].keys():
            if onlyNominal & ~variation.startswith("nominal"): continue
            #print(variation)
            file_list = file_info[process][variation]["files"]
            if n_files_max_per_sample != -1:
                file_list = file_list[:n_files_max_per_sample]  # use partial set of samples

            file_paths = [f["path"] for f in file_list]
            nevts_total = sum([f["nevts"] for f in file_list])
            metadata = {"process": process, "variation": variation, "nevts": nevts_total, "xsec": xsec_info[process]}
            fileset.update({f"{process}__{variation}": {"files": file_paths, "metadata": metadata}})

    return fileset

In [5]:
fileset = construct_fileset(N_FILES_MAX_PER_SAMPLE, dataset=DATA,
                            onlyNominal=True, ntuples_json='../data/ntuples_nanoaod_V2.json') 

print(f"processes in fileset: {list(fileset.keys())}")
print(f"\nexample of information in fileset:\n{{\n  'files': [{fileset['ttbar__nominal']['files'][0]}, ...],")
print(f"  'metadata': {fileset['ttbar__nominal']['metadata']}\n}}")
print(f"\nexample of data information in fileset:\n{{\n  'files': [{fileset['data']['files'][0]}, ...],")

processes in fileset: ['ttbar__nominal', 'single_top_tW__nominal', 'wjets__nominal', 'tttt__nominal', 'data']

example of information in fileset:
{
  'files': [https://xrootd-local.unl.edu:1094//store/user/AGC/nanoAOD/TT_TuneCUETP8M1_13TeV-powheg-pythia8/cmsopendata2015_ttbar_19980_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext3-v1_00000_0000.root, ...],
  'metadata': {'process': 'ttbar', 'variation': 'nominal', 'nevts': 11378043, 'xsec': 831.76}
}

example of data information in fileset:
{
  'files': [root://eosuser.cern.ch//eos/user/a/algomez/tmpFiles/opendata_files/SingleElectron/cmsopendata2015_Run2015D_SingleElectron_MINIAOD_08Jun2016-v1_00.root, ...],


## Analyzer

Here is the main analyzer. Uses coffea/awkward to make the analysis.

In [6]:
class fourTopAnalysis(processor.ProcessorABC):
    def __init__(self, DATASET):
        
        self.DATASET = DATASET
        
        #signal i.e. tttt
        self.n_jets_sig = []
        self.n_bjets_sig = []
        self.ht_sig = []
        self.htb_sig = []
        self.htratio_sig = []
        
        #tt
        self.n_jets_tt = []
        self.n_bjets_tt = []
        self.ht_tt = []
        self.htb_tt = []
        self.htratio_tt = []
        
        #background
        self.n_jets_bkg = []
        self.n_bjets_bkg = []
        self.ht_bkg = []
        self.htb_bkg = []
        self.htratio_bkg = []
        
        #data
        self.n_jets_data = []
        self.n_bjets_data = []
        self.ht_data = []
        self.htb_data = []
        self.htratio_data = []
       
        #### booking histograms
        ## define categories
        process_cat = hist.axis.StrCategory([], name="process", label="Process", growth=True)
        variation_cat  = hist.axis.StrCategory([], name="variation", label="Systematic variation", growth=True)
        
        ## define bins (axis)
        pt_axis = hist.axis.Regular( bins=500, start=0, stop=500, name="var")
        eta_axis = hist.axis.Regular( bins=40, start=-5, stop=5, name="var")
        num_axis = hist.axis.Regular( bins=40, start=1, stop=16 , name="var")
        num_bjets_axis = hist.axis.Regular( bins=6, start=1.5, stop=7.5 , name="var")
        
        ## define new variable bins
        HTb_axis = hist.axis.Regular( bins=200, start=50, stop=1000, name="var")
        HT_axis = hist.axis.Regular( bins=200, start=200, stop=2500, name="var")
        HTratio_axis = hist.axis.Regular( bins=500, start=0, stop=1, name="var")
        
        ## define a dictionary of histograms
        self.hist_muon_dict = {
            'ele_pt'   : (hist.Hist(pt_axis, process_cat, variation_cat, storage=hist.storage.Weight())),
            'ele_eta'  : (hist.Hist(eta_axis, process_cat, variation_cat, storage=hist.storage.Weight())),
            'neles'    : (hist.Hist(num_axis, process_cat, variation_cat, storage=hist.storage.Weight())),
            'jets_pt'  : (hist.Hist(pt_axis, process_cat, variation_cat, storage=hist.storage.Weight())),
            'jets_eta' : (hist.Hist(eta_axis, process_cat, variation_cat, storage=hist.storage.Weight())),
            'njets'    : (hist.Hist(num_axis, process_cat, variation_cat, storage=hist.storage.Weight())),
            'nbjets'   : (hist.Hist(num_bjets_axis, process_cat, variation_cat, storage=hist.storage.Weight())),
            'sumbjets_pt'   : (hist.Hist(HTb_axis, process_cat, variation_cat, storage=hist.storage.Weight())),
            'H_pt'   : (hist.Hist(HT_axis, process_cat, variation_cat, storage=hist.storage.Weight())),
            'Hratio_pt'   : (hist.Hist(HTratio_axis, process_cat, variation_cat, storage=hist.storage.Weight())),

        }
         
        sumw_dict = {'sumw': processor.defaultdict_accumulator(float)}
     

    def process(self, events ):

        hists = self.hist_muon_dict.copy()

        process = events.metadata["process"]  # "ttbar" etc.
    
        if process != "data":
            # normalization for MC
            x_sec = events.metadata["xsec"]
            nevts_total = events.metadata["nevts"]
            print(f'events {process}: {nevts_total}')
            #lumi = 2611.5 # /pb
            lumi = 2256.38
            xsec_weight = x_sec * lumi / nevts_total
        else:
            xsec_weight = 1
            print(f'events {process}')

        events["pt_nominal"] = 1.0

        ################
        #### Object Selection
        ################
        #### electrons. In this case veto_ele and selected_ele are not a subset since cutBased value is exclusive.
        
        #Veto electrons pT > 15 and |eta|<2.5
        veto_ele_selection = (events.Electron.pt > 15) & (abs(events.Electron.eta) < 2.5) & (events.Electron.cutBased == 1)
        
        # pT > 30 GeV and |eta|<2.1 for electrons (tight electrons)
        selected_ele_selection = (events.Electron.pt > 30) & (abs(events.Electron.eta) < 2.1) & (events.Electron.cutBased == 4)
        
        selected_electrons = events.Electron[ selected_ele_selection ]
        selected_electron = (ak.count(selected_electrons.pt, axis=1) ==1 ) #we need exactly 1 electron per event fits tight characteristis
        #print(selected_electrons) ### [ [electron1, electron2], [electron1], [], .....  ]
        veto_electrons = events.Electron[ veto_ele_selection ]
        veto_electron = (ak.count(veto_electrons.pt, axis=1) == 0 ) #we need exactly 0 electrons per event that fits veto characteristics
        
        
        '''
        ###comprobacion
        if process == 'tttt':
            N_counts = ak.num(selected_electrons.pt, axis=0)
            print(N_counts)
            
            nevts_total = events.metadata["nevts"]
            
            lumi = 2256.38
            xsec_weight = x_sec * lumi / nevts_total
            
            value = N_counts * xsec_weight
            print(value)
            
            value2= 821*lumi*1000
            print(value2)
        '''

        #pT > 30 GeV & |eta| < 2.5 for jets
        jet_selection = (events.Jet.pt * events["pt_nominal"] > 30) & (abs(events.Jet.eta) < 2.5) & (events.Jet.jetId > 1)
        selected_jets = events.Jet[jet_selection]
        nearest_electron = selected_jets.nearest(selected_electrons, threshold=0.4)
        # selected_jets contains both, jets and bjets
        selected_jets = selected_jets[ ~ak.is_none(nearest_electron) ]
        
        ## the results of these 2 lines should be equivalent to the 2 lines above
        #lepton_mask = ak.any(selected_jets.metric_table(selected_lepton, metric=lambda j, e: ak.local_index(j, axis=1) == e.jetIdx,), axis=2)
        #selected_jets = selected_jets[~lepton_mask]
        
        B_TAG_THRESHOLD = 0.8
        selected_bjets = events.Jet[jet_selection & ~ak.is_none(nearest_electron) & (events.Jet.btagCSVV2 >= B_TAG_THRESHOLD)]
        selected_jets_nobjets = events.Jet[jet_selection & ~ak.is_none(nearest_electron) & ~(events.Jet.btagCSVV2 >= B_TAG_THRESHOLD)] 
        
        ################
        #### Event Selection
        ################
        
        #Trigger
        event_filters = ( events.HLT.Ele23_WPLoose_Gsf == 1 )

        event_filters = event_filters & ( selected_electron & veto_electron )
        
        # at least six jets
        event_filters = event_filters & (ak.count(selected_jets.pt, axis=1) >= 6)
        
        # at least two b-tagged jets ("tag" means score above threshold)
        event_filters = event_filters & (ak.count(selected_bjets.pt, axis = 1) >= 2)
        
        # apply event filters
        selected_events = events[event_filters]
        selected_electrons = selected_electrons[event_filters]
        selected_jets = selected_jets[event_filters]
        selected_bjets = selected_bjets[event_filters]
        
        ###
        ##Now let's create the variable $H_T$ (that is the sum of the p_t of all jets in each event)
        total_pt = ak.sum(selected_jets.pt, axis=1)
        #print(total_pt)
        ###
        
        ###
        ## Now we create the needed variables for creating H_T^{ratio} variable
        ###
        jets_pt = selected_jets.pt
        ordered_jets = ak.sort(jets_pt, axis=1, ascending=False) #this array contains jets' pt ordered from the largest to the smallest value
 
        
        sorted_jets = []
        sumpt_remaining_jets = []
        if len(ordered_jets) > 0: #this if clause is to just take in account no empty arrays
            # the array sliced_jets contains just the first four highest values of jets' pt in each event(subarray)
            sliced_jets = ordered_jets[:,:4]
            #print("Sliced Jets")
            #print(sliced_jets)
            ## the array sorted_jets cotains the sum of the four highest values of jets' pt in the event
            sorted_jets = ak.sum(sliced_jets, axis=1)
            #print(sorted_jets)
            #remaining_jets contains the values of the remaining pt jets excluding the four highest values
            remaining_jets = ordered_jets[:,4:]
            #sumpt_reamaining_jets contains the sum of the remaining pt jets excluding the four highest values
            sumpt_remaining_jets = ak.sum(remaining_jets, axis=1)
        
        ##Finally we create the variable H_T^{ratio}
        Ht_ratio = [x / y for x, y in zip(sumpt_remaining_jets,sorted_jets)]
        #Ht_ratio2 = [x / y for x, y in zip(sorted_jets,sumpt_remaining_jets)]
        
        
            
        ###
        ### The next lines of code are used for creating a 'super' variable i.e. a variable similar to a BDT analysis 
        ### which follows some constrains or restrictions
        ###
        
        n_jets = ak.count(selected_jets.pt, axis=1) ##this is number of jets
        #print(n_jets)
        n_bjets = ak.count(selected_bjets.pt, axis=1) ## this is number of bjets  
        ht_bjets = ak.sum(selected_bjets.pt, axis=1)
                
        for ivar in [ "pt", "eta" ]:
            
            hists[f'ele_{ivar}'].fill(
                        var=ak.flatten(getattr(selected_electrons, ivar)), process=process,
                        variation="nominal", weight=xsec_weight
                    )
            hists[f'jets_{ivar}'].fill(
                        var=ak.flatten(getattr(selected_jets, ivar)), process=process,
                        variation="nominal", weight=xsec_weight
                    )
            
        hists['neles'].fill(
                    var=ak.count(selected_electrons.pt, axis=1), process=process,
                    variation="nominal", weight=xsec_weight
                )
        hists['njets'].fill(
                    var=n_jets, process=process,
                    variation="nominal", weight=xsec_weight
                )
        hists['nbjets'].fill(
                    var=n_bjets, process=process,
                    variation="nominal", weight=xsec_weight
                )
        hists['sumbjets_pt'].fill(
                    var= ht_bjets, process=process,
                    variation="nominal", weight=xsec_weight
                ) 
        hists['H_pt'].fill(
                    var= total_pt, process=process,
                    variation="nominal", weight=xsec_weight
                )
        hists['Hratio_pt'].fill(
                    var= Ht_ratio, process=process,
                    variation="nominal", weight=xsec_weight
                )
            
        if process == 'tttt':
            self.n_jets_sig.extend(n_jets)
            self.n_bjets_sig.extend(n_bjets)
            self.ht_sig.extend(total_pt)
            self.htb_sig.extend(ht_bjets)
            self.htratio_sig.extend(Ht_ratio)
            
        if process != 'data' and process != 'tttt':
            self.n_jets_bkg.extend(n_jets)
            self.n_bjets_bkg.extend(n_bjets)
            self.ht_bkg.extend(total_pt)
            self.htb_bkg.extend(ht_bjets)
            self.htratio_bkg.extend(Ht_ratio)
                
        if process == 'ttbar':
            self.n_jets_tt.extend(n_jets)
            self.n_bjets_tt.extend(n_bjets)
            self.ht_tt.extend(total_pt)
            self.htb_tt.extend(ht_bjets)
            self.htratio_tt.extend(Ht_ratio)
                
        if process == 'data':
            self.n_jets_data.extend(n_jets)
            self.n_bjets_data.extend(n_bjets)
            self.ht_data.extend(total_pt)
            self.htb_data.extend(ht_bjets)
            self.htratio_data.extend(Ht_ratio)
                

        output = {"nevents": {events.metadata["dataset"]: len(events)}, "hists" : hists
                    , "n_jets_sig": self.n_jets_sig
                    , "n_bjets_sig": self.n_bjets_sig
                    , "ht_sig": self.ht_sig
                    , "htb_sig": self.htb_sig
                    , "htratio_sig": self.htratio_sig
                      
                    , "n_jets_bkg": self.n_jets_bkg
                    , "n_bjets_bkg": self.n_bjets_bkg
                    , "ht_bkg": self.ht_bkg
                    , "htb_bkg": self.htb_bkg                     
                    , "htratio_bkg": self.htratio_bkg
                      
                    , "n_jets_data": self.n_jets_data
                    , "n_bjets_data": self.n_bjets_data
                    , "ht_data": self.ht_data
                    , "htb_data": self.htb_data
                    , "htratio_data": self.htratio_data
                      
                    , "n_jets_tt": self.n_jets_tt
                    , "n_bjets_tt": self.n_bjets_tt
                    , "ht_tt": self.ht_tt
                    , "htb_tt": self.htb_tt
                    , "htratio_tt": self.htratio_tt
                    }

        return output


    def postprocess(self, accumulator):
        
        return accumulator

Let's make it run:

In [7]:
#n_jets_data = []
executor = processor.FuturesExecutor(workers=NUM_CORES)

run = processor.Runner(executor=executor, schema=NanoAODSchema, 
                       savemetrics=True, metadata_cache={}, chunksize=CHUNKSIZE)

t0 = time.monotonic()

all_histograms, metrics = run(fileset, "Events", processor_instance = fourTopAnalysis(DATASET=DATA))
exec_time = time.monotonic() - t0


Preprocessing:   0%|          | 0/87 [00:00<?, ?file/s]

Processing:   0%|          | 0/112 [00:00<?, ?chunk/s]

events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events data
events tttt: 1210521
events tttt: 1210521
events tttt: 1210521
events tttt: 1210521
events tttt: 1210521
events tttt: 1210521
events tttt: 1210521
events wjets: 11817704
events wjets: 11817704
events wjets: 11817704
events wjets: 11817704
events wjets: 11817704
events wjets: 11817704
events wjets: 11817704
events w

In [8]:
#import pickle

with open("histograms.pkl", "wb") as f:
    pickle.dump(all_histograms["hists"], f, protocol=pickle.HIGHEST_PROTOCOL)
    

In [9]:
##Saving arrays so that we can use them without running the whole code again

##signal
n_jets_sig = np.array(all_histograms["n_jets_sig"])
n_bjets_sig = np.array(all_histograms["n_bjets_sig"])
ht_sig = np.array(all_histograms["ht_sig"])
htb_sig = np.array(all_histograms["htb_sig"])
htratio_sig = np.array(all_histograms["htratio_sig"])

np.save('n_jets_sig.npy', n_jets_sig )
np.save('n_bjets_sig.npy', n_bjets_sig )
np.save('ht_sig.npy', ht_sig )
np.save('htb_sig.npy', htb_sig )
np.save('htratio_sig.npy', htratio_sig )

##background
n_jets_bkg = np.array(all_histograms["n_jets_bkg"])
n_bjets_bkg = np.array(all_histograms["n_bjets_bkg"])
ht_bkg = np.array(all_histograms["ht_bkg"])
htb_bkg = np.array(all_histograms["htb_bkg"])
htratio_bkg = np.array(all_histograms["htratio_bkg"])

np.save('n_jets_bkg.npy', n_jets_bkg )
np.save('n_bjets_bkg.npy', n_bjets_bkg )
np.save('ht_bkg.npy', ht_bkg )
np.save('htb_bkg.npy', htb_bkg )
np.save('htratio_bkg.npy', htratio_bkg )

##data
n_jets_data = np.array(all_histograms["n_jets_data"])
n_bjets_data = np.array(all_histograms["n_bjets_data"])
ht_data = np.array(all_histograms["ht_data"])
htb_data = np.array(all_histograms["htb_data"])
htratio_data = np.array(all_histograms["htratio_data"])

np.save('n_jets_data.npy', n_jets_data )
np.save('n_bjets_data.npy', n_bjets_data )
np.save('ht_data.npy', ht_data )
np.save('htb_data.npy', htb_data )
np.save('htratio_data.npy', htratio_data )

##tt
n_jets_tt = np.array(all_histograms["n_jets_tt"])
n_bjets_tt = np.array(all_histograms["n_bjets_tt"])
ht_tt = np.array(all_histograms["ht_tt"])
htb_tt = np.array(all_histograms["htb_tt"])
htratio_tt = np.array(all_histograms["htratio_tt"])

np.save('n_jets_tt.npy', n_jets_tt )
np.save('n_bjets_tt.npy', n_bjets_tt )
np.save('ht_tt.npy', ht_tt )
np.save('htb_tt.npy', htb_tt )
np.save('htratio_tt.npy', htratio_tt )

In [11]:
dataset_source = "/data" if fileset["ttbar__nominal"]["files"][0].startswith("/data") else "https://xrootd-local.unl.edu:1094" # TODO: xcache support
metrics.update({"walltime": exec_time, "num_workers": NUM_CORES, "dataset_source": dataset_source, 
                "n_files_max_per_sample": N_FILES_MAX_PER_SAMPLE, 
                "cores_per_worker": CORES_PER_WORKER, "chunksize": CHUNKSIZE})#

print(f"event rate per worker (full execution time divided by NUM_CORES={NUM_CORES}): {metrics['entries'] / NUM_CORES / exec_time / 1_000:.2f} kHz")
print(f"event rate per worker (pure processtime): {metrics['entries'] / metrics['processtime'] / 1_000:.2f} kHz")
print(f"amount of data read: {metrics['bytesread']/1000**2:.2f} MB")  # likely buggy: https://github.com/CoffeaTeam/coffea/issues/717


event rate per worker (full execution time divided by NUM_CORES=4): 9.53 kHz
event rate per worker (pure processtime): 11.79 kHz
amount of data read: 2393.35 MB
